In [15]:
import pandas as pd
import folium
import ipywidgets as widgets  
from ipywidgets import SelectMultiple
from tabulate import tabulate

war_data = pd.read_csv(r'data/war_events.csv', usecols=['YEAR_MONTH', 'EVENT_TYPE', 'SUB_EVENT_TYPE', 'ADMIN1', 'LATITUDE', 'LONGITUDE', 'FATALITIES', 'NOTES'])

color_map = {'2019': '#DD2323',
             '2018': '#3186cc',
             '2020': '#1AAA27',
             '2021': '#B81CA7',
             '2022': '#B1B81C'}

war_data.YEAR_MONTH.unique()

array(['January 2018', 'February 2018', 'March 2018', 'April 2018',
       'May 2018', 'June 2018', 'July 2018', 'August 2018',
       'September 2018', 'October 2018', 'November 2018', 'December 2018',
       'January 2019', 'February 2019', 'March 2019', 'April 2019',
       'May 2019', 'June 2019', 'July 2019', 'August 2019',
       'September 2019', 'October 2019', 'November 2019', 'December 2019',
       'January 2020', 'February 2020', 'March 2020', 'April 2020',
       'May 2020', 'June 2020', 'July 2020', 'August 2020',
       'September 2020', 'October 2020', 'November 2020', 'December 2020',
       'January 2021', 'February 2021', 'March 2021', 'April 2021',
       'May 2021', 'June 2021', 'July 2021', 'August 2021',
       'September 2021', 'October 2021', 'November 2021', 'December 2021',
       'January 2022', 'February 2022', 'March 2022', 'April 2022',
       'May 2022', 'June 2022', 'July 2022', 'August 2022',
       'September 2022', 'October 2022'], dtype=object)

In [33]:
def generate_map(date: str, event_type: str, region: str) -> 'Subsected DataFrame':
    
    try:
        if date[0].lower() == 'all':
            date = war_data['YEAR_MONTH'].unique()
        if event_type[0].lower() == 'all':
            event_type = war_data.EVENT_TYPE.unique()
        if region[0].lower() == 'all':
            region = region.ADMIN1.unique()

        map_data = war_data[(war_data['YEAR_MONTH'].isin(date))
                        &(war_data['EVENT_TYPE'].isin(event_type))
                        &(war_data['ADMIN1'].isin(region))]
    except:
        return 'No Data For Selected Parameters'

    i_map = folium.Map(
            location=[war_data["LATITUDE"].mean(), war_data["LONGITUDE"].mean()-3.5],
            tiles='stamenterrain',
            zoom_start=5.5,
            zoom_control=False,
            scrollWheelZoom=False,
            dragging=False)

    map_data.apply(
        lambda row: folium.CircleMarker(
            location=[row['LATITUDE'], row['LONGITUDE']],
#             radius=row['FATALITIES']/20,
            tooltip='<h5>Click here for more info</h5>',
            popup=row['NOTES'],
            stroke=True,
            weight=1,
            color="#3186cc",
            fill=True,
            fill_color="#3186cc",
            opacity=0.9,
            fill_opacity=0.3,
            ).add_to(i_map),
        axis=1)
    
    grouped_war_data = map_data.groupby(['YEAR_MONTH', 'EVENT_TYPE', 'ADMIN1']).count().reset_index()
    grouped_war_data = grouped_war_data[grouped_war_data.columns[:4]]
    grouped_war_data.columns = [*grouped_war_data.columns[:-1], 'Count']
    print(tabulate(grouped_war_data.sort_values(by='Count', ascending=False),headers=[*grouped_war_data.columns], tablefmt='github', showindex=False))
    return i_map
    

search_terms = {'date': ['all'],
               'event_type': ['all'],
               'region': ['all']}

_= widgets.interact(
    generate_map, 
    date = SelectMultiple(options=war_data['YEAR_MONTH'].unique()),
    event_type = SelectMultiple(options=war_data['EVENT_TYPE'].unique()), 
    region = SelectMultiple(options=war_data['ADMIN1'].unique()),
)

# Color Code Year
# Add Average of Prices/Quantity Output Table into Interactive Narrative

interactive(children=(SelectMultiple(description='date', options=('January 2018', 'February 2018', 'March 2018…

In [ ]:
war